<a href="https://www.kaggle.com/code/aisuko/automatic-speech-recognition-asr?scriptVersionId=164769594" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Automatic speech recognition (ASR) converts a speech signal to text, mapping a sequence of audio inputs to text outputs. Many of virtual assistants like Siri and Alexa use ASR model to help users everyday, ans there are many other applications like live captioning and note-taking during meetings. Let's finetune Wav2Vec2-base model which pretrained on 16kHz sampled speech audio with `Automatic Speech Recognition` label dataset. We need to make sure the speech input is also sampled at 16kHz.

In [ ]:
%%capture
!pip install transformers==4.35.2
!pip install datasets==2.15.0
!pip install evaluate==0.4.1
!pip install jiwer==3.0.3

In [ ]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# checking Huggingface services status if the login was failed
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tuning Wav2Vec2-base"
os.environ["WANDB_NOTES"] = "Fine tune model distilbert base uncased"
os.environ["WANDB_NAME"] = "ft-wav2vec2-with-minds-asr"

# Loading MinDS-14


Let's pick up a smaller subset of the MInDS-14.

In [ ]:
from datasets import load_dataset, Audio

minds=load_dataset("PolyAI/minds14", name="en-US", split="train[:500]")

Split the dataset's train split into a train and test set with the `train_test_split` method

In [ ]:
minds=minds.train_test_split(test_size=0.2)
minds

Let's focus on the audio and transcription, and remove the other columns with the `remove_columns` method.

In [ ]:
minds=minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [ ]:
minds["train"][0]

There are two fileds:

* `audio`: a l-dimensional `array` of the speech signal that must be called to load and resample the audio file.
* `transcription`: the target text

# Preprocess

Let's load a Wav2Vec2 process to process the audio signal with `AutoProcessor` It is multimodel tasks require a processor that combines two types of preprocessing tools.

In [ ]:
from transformers import AutoProcessor

processor=AutoProcessor.from_pretrained("facebook/wav2vec2-base")

Let's resample the dataset to 16000kHz to use the pretrained Wav2Vec2 model:

In [ ]:
minds=minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

As we can see in the `transcription` above, the text contains a mix of upper and lowercase characters. The Wav2Vec2 **tokenizer** is only trained on uppercase characters. So we will need to make sure the text maches the tokenizer's vocabulary:

In [ ]:
def uppercase(example):
    return {"transcription": example["transcription"].upper()}

minds=minds.map(uppercase)

Let's create a preprocessing function that:
1. Calls the `audio` column to load and resample the audio file
2. Extracts the `input_values` from the audio file and tokenize the transcription column with the processor.

In [ ]:
def prepare_dataset(batch):
    audio=batch["audio"]
    batch=processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"]=len(batch["input_values"][0])
    return batch

encoded_minds=minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

We need to adapt the DataCollatorWithPadding to create a batch of examples. It will also dynamically pad our text and lables to the length of the longest element in its batch(instead of the entire dataset) so they are a uniform length. While it is possible to pad our text in the tokenizer function by setting padding=True, dynamic padding is more efficient.

In [ ]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str]="longest"
        
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need different padding methods
        input_features=[{"input_values": feature["input_values"][0]} for feature in features]
        label_features=[{"input_ids": feature["labels"]} for feature in features]
        
        batch=self.processor.pad(input_features, padding=self.padding, return_tensors="pt")
        
        labels_batch=self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")
        
        # replace padding with -100 to ignore loss correctly
        labels=labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        batch["labels"]=labels
        
        return batch

In [ ]:
data_collator=DataCollatorCTCWithPadding(processor=processor, padding="longest")
print(data_collator)

# Evaluate

Here we load a evaluation method with Evaluate library. In this notebook, load the word error rate metric:

In [ ]:
import evaluate
import numpy as np

wer_eva=evaluate.load("wer")


def compute_metrics(pred):
    pred_logits=pred.predictions
    pred_ids=np.argmax(pred_logits, axis=-1)
    
    pred.label_ids[pred.label_ids==-100]=processor.tokenizer.pad_token_id
    
    pred_str=processor.batch_decode(pred_ids)
    label_str=processor.batch_decode(pred.label_ids, group_tokens=False)
    
    wer=wer_eva.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

Then create a function that passes our predictions and labels to compute to calcualte the WER:

# Training

In [ ]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model=AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

print(model.config)

In [ ]:
training_args=TrainingArguments(
    output_dir=os.getenv("WANDB_NAME"),
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1500,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="steps",
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
    push_to_hub=False,   
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
import math

eval_results=trainer.evaluate()
print(f"Perplexity:{math.exp(eval_results['eval_loss']):2f}")

In [ ]:
kwargs={
    'model_name': f'{os.getenv("WANDB_NAME")}',
    'finetuned_from': "facebook/wav2vec2-base",
    'tasks': 'automatic speech recognition',
    'dataset_tags':'automatic speech recognition',
    'dataset':'PolyAI/minds14'
}

processor.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

# Inference

Load an audio file from dataset, and remember to resample the sampling rate of the audio file to match the sampling rate of the model.

In [ ]:
from datasets import load_dataset, Audio

dataset=load_dataset("PolyAI/minds14", "en-US", split="train")
dataset=dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate=dataset.features["audio"].sampling_rate
audio_file=dataset[0]["audio"]["path"]

In [ ]:
from transformers import pipeline

transcriber=pipeline("automatic-speech-recognition", model=os.getenv("WANDB_NAME"))
transcriber(audio_file)

## With PyTorch

In [ ]:
from transformers import AutoProcessor

processor=AutoProcessor.from_pretrained(os.getenv("WANDB_NAME"))
inputs=processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

In [ ]:
from transformers import AutoModelForCTC

model=AutoModelForCTC.from_pretrained(os.getenv("WANDB_NAME"))
with torch.no_grad():
    logits=model(**inputs).logits

In [ ]:
import torch

predicted_ids=torch.argmax(logits, dim=-1)
transcription=processor.batch_decode(predicted_ids)
transcription